In [75]:
import numpy as np
import matplotlib.pyplot as plt
import bm3d
from pathlib import Path
import csv
from skimage import io, img_as_float
import nibabel as nib
import os
import glob
from collections import defaultdict

File Paths

In [97]:
icip_data = "D:/VIP Cup/Dataset/ICIP training data/"
denoised_data = "D:/VIP Cup/Output dataset/denoised_tiff_data/"
denoised_nifti_dataset = "D:/VIP Cup/Output dataset/denoised_nifti_dataset"
in_path = "D:/VIP Cup/Dataset/ICIP training data/2/RawDataQA-2 (24)/RawDataQA-2-24 (108).tiff"
out_path = "D:/VIP Cup/Results/bm3d library/2-bm3d_sigma_"
csv_file = os.path.join(denoised_data, "tiff_directories.csv")

Constants

In [77]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.085

Functions to check whether a tiff file/directory or not

In [78]:
def is_tiff_dir(curr_path):
    '''function to check if the current directory having any tiff files'''
    num_tiff = len(glob.glob(os.path.join(curr_path, '*.tiff')))
    return num_tiff

def is_tiff(curr_path):
    '''function to check if the current file is a tiff file or not'''
    if os.path.isfile(curr_path) and (".tiff" in curr_path):
        return True
    else:
        return False

Searching tiff directories

In [80]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.083

dataset = glob.iglob(os.path.join(icip_data, '**'), recursive=True)

for i,file in enumerate(dataset):
    if is_tiff_dir(file):

        current_tiff_count = is_tiff_dir(file)
        tiff_directories[file] = current_tiff_count
        print(f"{len(tiff_directories)}th case {file} ......")
        original_tiff_count += current_tiff_count

with open(csv_file, 'w', newline='') as csvfile:
        
    writer = csv.writer(csvfile)
    #header
    writer.writerow(['Case', 'tiff files'])
    #write rows
    for key, value in tiff_directories.items():
        writer.writerow([key, value])

print(f"Denoised cases: {len(tiff_directories)}")
#print(f"Denoised tiff images: {denoised_tiff_count}")
print(f"Original tiff images: {original_tiff_count}")


1th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1) ......
2th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (10) ......
3th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (11) ......
4th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (12) ......
5th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13) ......
6th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (14) ......
7th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (15) ......
8th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (16) ......
9th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (17) ......
10th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (18) ......
11th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (19) ......
12th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (2) ......
13th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (20) ......
14th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (21) ......
15t

In [81]:
tiff_directories

defaultdict(int,
            {'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (1)': 52,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (10)': 273,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (11)': 300,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (12)': 258,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (13)': 146,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (14)': 220,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (15)': 87,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (16)': 52,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (17)': 176,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (18)': 300,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (19)': 84,
             'D:/VIP Cup/Dataset/ICIP training data\\0\\RawDataQA (2)': 201,
             'D:/VIP Cup/Dataset/ICIP training data\\

Denoising Function

In [82]:
def bm3d_denoise(noisy_image, npsd_sigma = SIGMA):

    noisy_oct = img_as_float(io.imread(noisy_image, as_gray=True))
    sigma = npsd_sigma
    bm3d_denoised = bm3d.bm3d(noisy_oct, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    return bm3d_denoised

Saving Denoised Image

In [83]:
def save_denoised_image(input_path, output_path):
    
    plt.imsave(output_path,input_path,  cmap='gray')

Denoising and Volumetric image conversion for one directory

In [95]:
curr_dir = list(tiff_directories.keys())[4]

img_dir = curr_dir
stack_axis = 2
    
fns = list([str(oct_image) for oct_image in glob.glob(os.path.join(img_dir,'*.tiff*'))])

case_no = lambda x: int(x.split('(')[-1].split(')')[0])
oct_images = sorted(fns, key = lambda y: case_no(y))

images = []

for oct_image in oct_images:
    print(oct_image)

    denoised_image = bm3d_denoise(oct_image,SIGMA)
    #save_out =  os.path.join("D:/VIP Cup/Results/bm3d_one_dir/", os.path.basename(oct_image))
    #save_denoised_image(denoised_image, save_out)

    denoised_image = np.asarray(denoised_image).astype(np.float64).squeeze()

    #img = np.asarray(Image.open(oct_image)).astype(np.float32).squeeze()
    if denoised_image.ndim != 2:
        raise Exception(f'Only 2D data supported. File {denoised_image} has dimension {denoised_image.ndim}.')
    else:
        images.append(denoised_image)

to_nifti = np.stack(images, stack_axis)
nib.Nifti1Image(to_nifti,None).to_filename("D:/VIP Cup/Results/bm3d_0_13_s=0.085_wrapper_float64.nii.gz")


D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (1).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (2).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (3).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (4).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (5).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (6).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (7).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (8).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (9).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (10).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (11).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (12).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)\RawDataQA13 (13).tiff
D:/VIP C

Output path generating function

In [98]:
def out_path(in_path, output_dir):
    '''generate output path'''
    base_name = os.path.basename(in_path)
    class_name = os.path.basename(os.path.dirname(in_path))
    out_path = os.path.join(output_dir,class_name) + "/" + base_name + ".nii.gz"
    return out_path

Denoising and Volumetric image conversion loop

In [101]:
denoised_nifti_dataset = "D:/VIP Cup/Output dataset/denoised_nifti_dataset"

for key, value in tiff_directories.items():

    img_dir = key
    print(key)

    fns = list([str(oct_image) for oct_image in glob.glob(os.path.join(img_dir,'*.tiff*'))])

    case_no = lambda x: int(x.split('(')[-1].split(')')[0])
    oct_images = sorted(fns, key = lambda y: case_no(y))

    images = []

    for oct_image in oct_images:
        #print(oct_image)

        denoised_image = bm3d_denoise(oct_image,SIGMA)
        #save_out =  os.path.join("D:/VIP Cup/Results/bm3d_one_dir/", os.path.basename(oct_image))
        #save_denoised_image(denoised_image, save_out)

        denoised_image = np.asarray(denoised_image).astype(np.float64).squeeze()

        #img = np.asarray(Image.open(oct_image)).astype(np.float32).squeeze()
        if denoised_image.ndim != 2:
            raise Exception(f'Only 2D data supported. File {denoised_image} has dimension {denoised_image.ndim}.')
        else:
            images.append(denoised_image)

    stack_axis = 2
    to_nifti = np.stack(images, stack_axis)
    out_name = out_path(img_dir, denoised_nifti_dataset)
    nib.Nifti1Image(to_nifti,None).to_filename(out_name)




D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (10)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (11)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (12)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (14)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (15)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (16)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (17)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (18)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (19)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (2)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (20)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (21)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (22)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (23)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (24)
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (25)
D:/VIP Cup/D